## Tutorial: Sentiment analysis de críticas de películas con regresión logística

#### Importamos las librerías necesarias

In [172]:
import pandas as pd
import numpy as np
import json
import re
import eli5
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import ToktokTokenizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

#### Cargamos los datos y creamos un Dataframe con el texto de las críticas, la nota y la URL

In [151]:
# Cargamos el fichero con las criticas y su puntuación
df = pd.read_csv("./data/criticas.csv")

# Mostramos las primeras 5 observaciones
df.head()

,critica,nota,url
0,"Bueno, bajo mi gusto, otro fracaso más de DC. ...",3,https://www.filmaffinity.com/es/reviews/1/4208...
1,Es tan terrible que podría funcionar como paro...,1,https://www.filmaffinity.com/es/reviews/1/4208...
2,Tengo una tradición desde hace más de 5 años. ...,2,https://www.filmaffinity.com/es/reviews/1/4208...
3,No entiendo como nadie tiene la cara de presen...,1,https://www.filmaffinity.com/es/reviews/1/4208...
4,La primera entrega de Wonder Woman (2017) no m...,4,https://www.filmaffinity.com/es/reviews/1/4208...


In [152]:
df.shape

(4800, 3)

#### Creamos una variable con el sentimiento a partir de la nota

In [153]:
# Creamos una variable con el sentimiento
# Si puntuación > 6 -> 1
# Si puntuación < 5 -> 0
df['sentiment'] = np.where(df['nota'] > 6, 1, 0)

In [154]:
# Eliminamos las variables nota y url
df.drop(columns=["nota","url"], inplace=True)

df.head()

,critica,sentiment
0,"Bueno, bajo mi gusto, otro fracaso más de DC. ...",0
1,Es tan terrible que podría funcionar como paro...,0
2,Tengo una tradición desde hace más de 5 años. ...,0
3,No entiendo como nadie tiene la cara de presen...,0
4,La primera entrega de Wonder Woman (2017) no m...,0


#### Limpiamos y preprocesamos el texto
Creamos un método que se encarge de realizar una limpieza inicial de los textos, tokenizar para dividir los textos en palabras individuales, filtrar las stopwords y los dígitos y realizar Stemming

In [155]:
tokenizer = ToktokTokenizer() 
STOPWORDS = set(stopwords.words("spanish"))
stemmer = SnowballStemmer("spanish")

def limpiar_texto(texto):
    """
    Función para realizar la limpieza de un texto dado.
    """
    # Eliminamos los caracteres especiales
    texto = re.sub(r'\W', ' ', str(texto))
    # Eliminado las palabras que tengo un solo caracter
    texto = re.sub(r'\s+[a-zA-Z]\s+', ' ', texto)
    # Sustituir los espacios en blanco en uno solo
    texto = re.sub(r'\s+', ' ', texto, flags=re.I)
    # Convertimos textos a minusculas
    texto = texto.lower()
    return texto

def filtrar_stopword_digitos(tokens):
    """
    Filtra stopwords y digitos de una lista de tokens.
    """
    return [token for token in tokens if token not in STOPWORDS 
            and not token.isdigit()]

def stem_palabras(tokens):
    """
    Reduce cada palabra de una lista dada a su raíz.
    """
    return [stemmer.stem(token) for token in tokens]

def tokenize(texto):
    """
    Método encargado de realizar la limpieza y preprocesamiento de un texto
    """
    text_cleaned = limpiar_texto(texto)
    tokens = [word for word in tokenizer.tokenize(text_cleaned) if len(word) > 1]
    tokens = filtrar_stopword_digitos(tokens)
    stems = stem_palabras(tokens)
    return stems

#### Dividimos el conjunto de datos en train y test

In [156]:
# Dividimos los datos en train y test
X = df.critica
y = df.sentiment
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, 
                                                    stratify=y, random_state=42)

In [157]:
# Comprobamos que train y test contienen el mismo porcentaje de cada clase en y
print(len(y_train[y_train == 0]) / 3224)
len(y_test[y_test == 0]) / 1588

0.4987593052109181


0.4987405541561713

#### Creamos el Vectorizador TFIDF y lo ajustamos con los datos de entrenamiento

In [158]:
tfidf = TfidfVectorizer(
    tokenizer=tokenize,
    max_features=20000)  

tfidf.fit(X_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=20000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x00000130FA8E0950>, use_idf=True,
        vocabulary=None)

#### Transformamos y aplicamos el Vectorizador a los conjuntos train y test 

In [159]:
X_train = tfidf.transform(X_train)
X_test = tfidf.transform(X_test)
print(X_train.shape)
print(X_test.shape)

(3216, 20000)
(1584, 20000)


#### Definimos los hiperparámetros, el modelo y GridSearchCV para buscar la mejor combinación de parámetros

In [160]:
# Objecto KFold para dividir un conjunto de datos en 10 bloques
cv = KFold(n_splits=10, shuffle=True, random_state=42)

# Diccionario con nombres de parámetros como claves y listas 
# de ajustes de parámetros a probar como valores
parameters = {'penalty':('l1', 'l2'), 'C':[100, 10, 1.0, 0.1, 0.01]}

# Modelo de Regresión logistica
lr = LogisticRegression(random_state=42, solver='liblinear')

# GridSearchCV para la búsqueda de los mejores parámetros
clf = GridSearchCV(lr, parameters, 
                   scoring='accuracy',
                   cv=cv,
                   refit=True,
                   verbose=2,
                   n_jobs=-1)

#### Ajustamos el GridSearchCV usando validación cruzada para la evaluación

In [161]:
clf.fit(X_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done  85 out of 100 | elapsed:    6.4s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    6.5s finished


GridSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
       error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'penalty': ('l1', 'l2'), 'C': [100, 10, 1.0, 0.1, 0.01]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=2)

In [162]:
print('Mejor combinación de parámetros: %s ' % clf.best_params_)
print('CV Accuracy: %.3f' % clf.best_score_)

Mejor combinación de parámetros: {'C': 10, 'penalty': 'l2'} 
CV Accuracy: 0.864


#### Obtenemos el mejor modelo mediante best_estimator_ y realizamos la evaluación con los datos de test

In [163]:
best_clf = clf.best_estimator_
print('Test Accuracy: %.3f' % best_clf.score(X_test, y_test))

Test Accuracy: 0.859


#### Mostramos el impacto de cada atributo (palabra) en las predicciones de cada clase

In [164]:
eli5.show_weights(estimator=best_clf, 
                  feature_names= list(tfidf.get_feature_names()),
                 top=(10, 10))

Weight?,Feature
+5.759,excelent
+4.899,cad
+4.757,reflexion
+4.676,perfect
+4.444,famili
+4.377,mund
+4.370,tempor
+4.323,maravill
+4.252,mejor
+3.987,magnif


#### Mostramos la matriz de confusión

In [182]:
y_pred = best_clf.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[675, 117],
       [107, 685]], dtype=int64)